# Customer Segmentation with RFM Analysis

### Business Problem:

The UK-based retail company wants to segment its customers and determine marketing strategies according to these segments.

RFM analysis will be used for segmentation.

The dataset named Online Retail II includes online sales transactions of a UK-based retail company between 01/12/2010 and 09/12/2011. The company's product catalog includes souvenirs and it is known that most of its customers are wholesalers.

InvoiceNo: Invoice Number (If this code starts with C, it means that the transaction has been cancelled) <br>
StockCode: Product code (unique for each product) <br>
Description: Product name <br>
Quantity: Number of products (How many of the products on the invoices were sold) <br>
InvoiceDate: Invoice date <br>
UnitPrice: Invoice price <br>
CustomerID: Unique customer number <br>
country: country name <br>

### 1. Importing the Libraries and Uploading Dataset

In [1]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df_ = pd.read_excel("online_retail.xlsx", sheet_name="Year 2010-2011")

In [3]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


### 2. General Information About Dataset

In [4]:
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

###############################    Shape  ##################################
(541910, 8)
###############################    Types  ##################################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
###############################    Head   ##################################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00  2.550    17850.000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00  3.390    17850.000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00  2.750    17850.000  United Kingdom
3  536365  

#### 2.1 Let's remove the missing observations from the dataset:

NA amount:

Description      1454

Customer ID    135080


In [5]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

#### 2.1 How many unique items we have in the dataset

In [6]:
df["Description"].nunique()

3896

#### 2.2 How many of each product are there?

In [7]:
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: Description, Length: 3896, dtype: int64

#### 2.3 The 'C' in the invoices shows the canceled transactions. Remove the canceled transactions from the dataset

In [8]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.500,14527.000,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.650,15311.000,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.650,17548.000,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom


In [9]:
df = df[~df["Invoice"].str.contains("C", na=False)] # removing canceled transactions

#### 2.4 Creating a variable named 'TotalPrice' that represents the total earnings per invoice

In [10]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340


### 3. Calculating RFM Metrics

The expansion of the word RFM corresponds to the meanings of recency, frequency, monetary.

RFM analysis helps firms reasonably predict which customers are likely to purchase their products again, how much revenue comes from new (versus repeat) clients, and how to turn occasional buyers into habitual ones.

Recency: How recently a customer has made a purchase
Frequency: How often a customer makes a purchase
Monetary Value: How much money a customer spends on purchases

In [11]:
# to calculate recency I give an analyse date = max invoice date + 2
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

#### 3.1 Let's calculate the customer-specific Recency, Frequency and Monetary metrics with groupby, agg and lambda and assign them to a variable called rfm

In [12]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,327,1,77183.600
12347.000,3,7,4310.000
12348.000,76,4,1797.240
12349.000,20,1,1757.550
12350.000,311,1,334.400


#### 3.2 Let's change the names of the metrics we created to recency, frequency and monetary

In [13]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4339.000,93.518,100.010,2.000,19.000,52.000,143.000,375.000
frequency,4339.000,4.272,7.705,1.000,1.000,2.000,5.000,210.000
monetary,4339.000,2053.797,8988.248,0.000,307.245,674.450,1661.640,280206.020


In [14]:
rfm[rfm["monetary"] == 0] # we have a 0 monetary value. Before continue, delete this.

,recency,frequency,monetary
Customer ID,,,
13256.000,15,1,0.000


In [15]:
rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4338.000,93.536,100.014,2.000,19.000,52.000,143.000,375.000
frequency,4338.000,4.273,7.706,1.000,1.000,2.000,5.000,210.000
monetary,4338.000,2054.271,8989.230,3.750,307.415,674.485,1661.740,280206.020


### 4. Generating RFM Scores and Converting to a Single Variable

Let's convert the Recency, Frequency and Monetary metrics to scores between 1-5 with the help of qcut.

In [16]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1]) # for recency highest values should be 1
rfm.head()

,recency,frequency,monetary,recency_score
Customer ID,,,,
12346.000,327,1,77183.600,1
12347.000,3,7,4310.000,5
12348.000,76,4,1797.240,2
12349.000,20,1,1757.550,4
12350.000,311,1,334.400,1


In [17]:
# needed to use rank for qcut operation.
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score
Customer ID,,,,,
12346.000,327,1,77183.600,1,1
12347.000,3,7,4310.000,5,5
12348.000,76,4,1797.240,2,4
12349.000,20,1,1757.550,4,1
12350.000,311,1,334.400,1,1


In [18]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.000,327,1,77183.600,1,1,5
12347.000,3,7,4310.000,5,5,5
12348.000,76,4,1797.240,2,4,4
12349.000,20,1,1757.550,4,1,4
12350.000,311,1,334.400,1,1,2


#### 4.1 Express recency_score and frequency_score as a single variable and save it as RF_SCORE

In RFM segmentation we use only Recency and Frequency metrics

In [19]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12346.000,327,1,77183.600,1,1,5,11
12347.000,3,7,4310.000,5,5,5,55
12348.000,76,4,1797.240,2,4,4,24
12349.000,20,1,1757.550,4,1,4,41
12350.000,311,1,334.400,1,1,2,11


#### 4.2 Let's define segments for the generated RF scores

In [20]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [21]:
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.000,327,1,77183.600,1,1,5,11,hibernating
12347.000,3,7,4310.000,5,5,5,55,champions
12348.000,76,4,1797.240,2,4,4,24,at_Risk
12349.000,20,1,1757.550,4,1,4,41,promising
12350.000,311,1,334.400,1,1,2,11,hibernating


#### 4.3 Let's see segments results

In [22]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").mean().sort_values("monetary",ascending = False)

,recency,frequency,monetary
segment,,,
champions,6.877,12.417,6857.964
loyal_customers,34.469,6.458,2856.720
cant_loose,133.429,8.381,2796.156
at_Risk,156.062,2.866,1076.506
potential_loyalists,18.124,2.010,1034.905
need_attention,54.065,2.328,889.226
hibernating,218.898,1.101,487.708
about_to_sleep,54.504,1.162,461.062
new_customers,7.857,1.000,388.213


In [23]:
rfm[rfm["segment"] == "champions"] # our champions :)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12347.000,3,7,4310.000,5,5,5,55,champions
12362.000,4,10,5226.230,5,5,5,55,champions
12364.000,9,4,1313.100,5,4,4,54,champions
12381.000,6,5,1845.310,5,4,4,54,champions
12417.000,4,9,3649.100,5,5,5,55,champions
...,...,...,...,...,...,...,...,...
18241.000,11,17,2073.090,5,5,5,55,champions
18245.000,8,7,2567.060,5,5,5,55,champions
18272.000,4,6,3078.580,5,5,5,55,champions


Hope it will be useful :)

linkedin.com/in/demir-zumrut/